In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import holidays
from scipy.optimize import newton
pd.set_option('display.max_rows', 500)


In [2]:
def get_accrual_date(date, holidays_req):
    # Check if the date is a weekend or holiday
    next_month = date.replace(day=28) + timedelta(days=4) 
    month_end  = next_month - timedelta(days=next_month.day)
    
    while date in holidays_req or date.weekday() >= 5:
        if date.weekday() >= 5: ### check if its a weekend
            if date == month_end: ### check if date is on a month end
                date -= timedelta(days=date.weekday()%4)
            else: 
                date += timedelta(days=7-date.weekday())
        else: ### if a day is a holiday
            date += timedelta(days=1)
    return date

def add_business_days(start_date, num_business_days, holiday_dates):
    start_date = np.datetime64(pd.to_datetime(start_date).date(), 'D')
    holiday_dates = np.array([np.datetime64(date, 'D') for date in holiday_dates.keys()], dtype='datetime64[D]')
    result_date = np.busday_offset(start_date, num_business_days, holidays=holiday_dates)
    return pd.Timestamp(result_date)

In [3]:
class yield_swaps:
    def __init__(self,data,pricing_date,settlement_date):
        self.df = data
        self.pricing_date = pd.Timestamp(pricing_date)
        self.holidays = holidays.US()
        self.DFs = {}
        self.Forwards = {}

    def Discount_Factor(self,date_req):
        date_obj = pd.Timestamp(date_req)
        start = pd.Timestamp(self.pricing_date)
        rem_days = (date_obj - start).days

        fwd_inst_keys = [start] + list(self.Forwards.keys())
        filtered_fwds = [date for date in fwd_inst_keys if date < date_obj]
        if len(filtered_fwds) < len(fwd_inst_keys):
            filtered_fwds.append(date_obj)
            shifted_days = list(pd.Series(filtered_fwds).diff().dt.days)[1:]
        else:
            shifted_days = list(pd.Series(filtered_fwds).diff().dt.days)[1:]
            shifted_days[-1] += (date_obj - fwd_inst_keys[-1]).days
            
        fwd_inst_vals = list(self.Forwards.values())[:len(shifted_days)]
        wavg = np.sum(np.array(fwd_inst_vals)*np.array(shifted_days))
        return np.exp(-wavg/360)
    
    def float_leg(self,acc_start,acc_end,FV=1):
        ### For now returning FV, have to update the handling
        DF_start = self.Discount_Factor(acc_start)
        DF_end = self.Discount_Factor(acc_end)
        return FV*(DF_start/DF_end - 1)

    def fixed_leg(self,swap_rate,acc_start,acc_end,FV=1):
        cal_days = (acc_end - acc_start).days
        coupon = FV*0.01*swap_rate*cal_days/360 ### Can be treated as coupon payment
        return coupon

    def ytm_solver(self, terms, fixed_pay, coupon_diff =0, guess=0.05):
        Forwards = np.array(list(self.Forwards.values())) ### Fitted forwards so far
        # Define the function for which we want to find the root
        def func(fc):
            ### below is for the payment involving the variable DF
            floater = 0
            if len(Forwards):
                terms_fitted = np.array(terms[:len(Forwards)])
                avg_fitted = np.sum(Forwards*terms_fitted)/360
                avg_variable = fc*terms[-1]/360
                avg_curr = avg_fitted + avg_variable
                return 1 - ((1+fixed_pay)/np.exp(avg_curr))
            else:
                avg_curr = fc*terms[0]/360
                return 1 - ((1+fixed_pay)/np.exp(avg_curr))
        ytm_value = newton(func, guess)
        return ytm_value
        
    def ytm_solver_yearly(self,instrument):
        fitted_fwds_max = max(list(self.Forwards.keys()))
        fitted_coupons = [coupon for coupon in instrument.coupon_accruals if coupon <= fitted_fwds_max ]
        fixed_pay = float_pay = 0
        start = self.pricing_date
        
        for i in range(len(fitted_coupons)):
            pay_date = add_business_days(fitted_coupons[i],2,self.holidays)
            fixed_pay += self.fixed_leg(instrument.mid,start,fitted_coupons[i])*self.Discount_Factor(pay_date)
            float_pay += self.float_leg(start,fitted_coupons[i])*self.Discount_Factor(pay_date)
            start = fitted_coupons[i]
        
        fitted_coupon_diff = float_pay - fixed_pay
        DF_last_Fit = self.Discount_Factor(list(self.Forwards.keys())[-1])
        DF_last_coup = self.Discount_Factor(fitted_coupons[-1])
        fixed_pay = self.fixed_leg(instrument.mid,fitted_coupons[-1],instrument.accrual_end)
        
        def func(fc):
            fc_small = np.exp(-fc/360)
            fixed_pay = float_pay = 0
            last_fitted_date = list(self.Forwards.keys())[-1]
            all_payments = instrument.coupon_accruals + [instrument.accrual_end]
            unavail_coupons = [coupon for coupon in all_payments if coupon > fitted_fwds_max]
            DF_last_Fit = self.Discount_Factor(list(self.Forwards.keys())[-1])
            coup_start = fitted_coupons[-1]
            DF_last_coup = self.Discount_Factor(coup_start)
            
            for i in range(len(unavail_coupons)):
                pay_date = add_business_days(unavail_coupons[i],2,self.holidays)
                accrual_period = (unavail_coupons[i] - last_fitted_date).days
                payment_period = (pay_date - last_fitted_date).days
                DF_var_acc = DF_last_Fit*(fc_small**accrual_period)
                DF_var_pay = DF_last_Fit*(fc_small**payment_period)
                fixed_pay += self.fixed_leg(instrument.mid,coup_start,unavail_coupons[i])*DF_var_pay
                float_pay += DF_var_pay*(DF_last_coup/DF_var_acc - 1)
                coup_start = unavail_coupons[i]
                DF_last_coup = DF_var_acc
            total_diff = float_pay - fixed_pay + fitted_coupon_diff
            return total_diff
        
        ytm_value = newton(func,0.05)
        return ytm_value
        
    def infer_req_dates(self):
        df = self.df.copy()
        def f(x):
            if x["Unit"] == "WK":
                accrual = self.pricing_date + timedelta(weeks=x["Term"])
            if x["Unit"] == "MO":
                accrual = self.pricing_date + relativedelta(months=x["Term"])
            if x["Unit"] == "YR":
                accrual = self.pricing_date + relativedelta(years=x["Term"])
            accrual = get_accrual_date(accrual, self.holidays)
            Tenor = str(x["Term"]) + x["Unit"][0]
            payment = add_business_days(accrual,2,self.holidays)
            if x["Unit"] == "YR" or x["Term"] == 18:
                fwd_jump = payment
            else:
                fwd_jump = accrual
                
            return [accrual,payment,Tenor, fwd_jump]
        
        self.df[['accrual_end', 'payment_date', 'Tenor', 'fwd_end']] = df.apply(lambda x: pd.Series(f(x)), axis=1)
        piece_lengths = list(self.df['fwd_end'].diff().dt.days)
        piece_lengths[0] = 7.0
        self.df["fwd_period"] = pd.Series(piece_lengths)
        self.df["fwd_start"] = self.df.apply(lambda x: x["fwd_end"] - timedelta(x["fwd_period"]),axis=1)
        self.df["accrual_offset"] = self.df.apply(lambda x: (x["fwd_end"] - x["accrual_end"]).days,axis=1)

        def coupon_func(x):
            if x["Unit"] != "YR" and x["Term"] != 18:
                return []
            elif x["Term"] == 18:
                return [self.df.iloc[8]["accrual_end"]]
            else:
                accrual_list = []
                for i in range(1,x["Term"]):
                    coupon_accrual = self.pricing_date + relativedelta(years = i)
                    coupon_accrual = get_accrual_date(coupon_accrual,self.holidays)
                    accrual_list.append(coupon_accrual)
                return accrual_list
        self.df["coupon_accruals"] = self.df.apply(lambda x: coupon_func(x), axis = 1)
    
    def fit_fc(self):
        data = self.df
        time_periods = list(self.df['fwd_period'])
        for i in range(len(data)):
            instrument = data.iloc[i]
            if instrument.Unit != "YR" and instrument.Term != 18:
                fixed_pay = self.fixed_leg(instrument.mid,self.pricing_date,instrument.accrual_end)
                self.Forwards[instrument.accrual_end] = self.ytm_solver(time_periods[:i+1],fixed_pay)
            else:
                self.Forwards[instrument.fwd_end] = self.ytm_solver_yearly(instrument)
        data["Forwards"] = pd.Series(self.Forwards.values())
        return data

In [4]:
SOFR_rates = pd.read_excel("data/sofr_rates_input_020223.xlsx")
SOFR_rates["mid"] = 0.5*(SOFR_rates["Bid"] + SOFR_rates["Ask"])
SOFR_rates = SOFR_rates[["mid", "Term", "Unit" ]]
pricing_date = datetime(2023, 2, 6)
swap_anal = yield_swaps(SOFR_rates,pricing_date,pricing_date)
swap_anal.infer_req_dates()
fitted = swap_anal.fit_fc()

In [5]:
SOFR_TS = pd.read_excel("data/sofr_rates_output_020223.xlsx")

In [6]:
SOFR_TS = SOFR_TS.copy()
SOFR_TS["Fitted_DF"] = SOFR_TS["Date"].apply(lambda x: swap_anal.Discount_Factor(x))
zeros, forwards = [],[]
for i in range(0,len(SOFR_TS)-1):
    x = SOFR_TS.iloc[i]
    x_next = SOFR_TS.iloc[i+1]
    zeros.append( -100*np.log(x_next["Fitted_DF"])*365/(pd.Timestamp(x_next["Date"])-pd.Timestamp(pricing_date)).days )
    forwards.append( -100*np.log(x_next["Fitted_DF"]/x["Fitted_DF"])*365/(pd.Timestamp(x_next["Date"])-pd.Timestamp(x["Date"])).days)
zeros = ["NaN"] + zeros ### zero is NA for the first date as its the same day
forwards = forwards + [forwards[-1]] ### forwards for last date would be same as previous periods as those are the only instruments we have
SOFR_TS["Fitted_Zeros"] = pd.Series(zeros)
SOFR_TS["Fitted_Fwds"] = pd.Series(forwards)
SOFR_TS["DF_diff"] = SOFR_TS["Fitted_DF"] - SOFR_TS["DF"]
SOFR_TS["Zeros_diff"] = SOFR_TS["Fitted_Zeros"] - SOFR_TS["Zero rate"]
SOFR_TS["Forward_diff"] = SOFR_TS["Fitted_Fwds"] - SOFR_TS["Forward"]

In [7]:
SOFR_TS = SOFR_TS[["Date", "Fitted_DF", "Fitted_Zeros", "Zero rate", "Fitted_Fwds", "Forward", "Zeros_diff", "Forward_diff" ]]
SOFR_TS

,Date,Fitted_DF,Fitted_Zeros,Zero rate,Fitted_Fwds,Forward,Zeros_diff,Forward_diff
0,2023-02-06,1.000000,NaN,NaN,4.829605,4.829605,NaN,-2.630784e-11
1,2023-08-06,0.976335,4.829605,4.829605,4.689564,4.689564,-0.0,1.279687e-11
2,2024-02-06,0.953525,4.759009,4.759009,3.803367,3.803367,0.0,-1.055023e-11
3,2024-08-06,0.935612,4.441044,4.441044,3.037370,3.037370,0.0,-8.415046e-12
4,2025-02-06,0.921395,4.087725,4.087725,2.707703,2.707703,-0.0,1.704414e-11
5,2025-08-06,0.909106,3.813839,3.813839,2.700444,2.700444,0.0,-4.651479e-11
6,2026-02-06,0.896814,3.626919,3.626919,2.662078,2.662078,0.0,4.920064e-11
7,2026-08-06,0.885053,3.490164,3.490164,2.661211,2.661211,0.0,-4.559908e-12
8,2027-02-06,0.873259,3.385765,3.385765,2.700659,2.700659,-0.0,1.840128e-11
9,2027-08-06,0.861642,3.310244,3.310244,2.701550,2.701550,0.0,-3.361844e-11
